In [3]:
import numpy as np
import numpy.linalg as la
import pandas as pd
import geopandas as gpd
from tqdm.notebook import tqdm

import osmnx as ox
from matplotlib import pyplot as plt
import momepy
import networkx as nx
from shapely.geometry import Polygon
import alphashape
from pyproj import Proj, Geod
import ast

data_path = '../../data/'  
df_car_detectors = gpd.read_file(data_path + 'all_car_detectors.geojson')

# polygon_road_network = gpd.read_file(data_path + 'network/QGIS_Project/referentiel-comptages-edit.shp')
# paris_districts = gpd.read_file(data_path + 'districts_paris.geojson')
# cycleways = gpd.read_file(data_path + 'cycleways_2000-2022.geojson')

# output_path = "output/"

In [4]:
df_car_detectors

,iu_ac,date_debut,date_fin,libelle,iu_nd_aval,libelle_nd_aval,iu_nd_amont,libelle_nd_amont,geo_point_2d,geometry
0,1743,1996-10-03 02:00:00+00:00,2023-01-01 01:00:00+00:00,Bd_Macdonald,928,Bd_Macdonald - Jaques Duchesne,929,Bd_Macdonald - Lounes Matoub,"{'lon': 2.3753362643387073, 'lat': 48.89885056...","LINESTRING (2.37633 48.89887, 2.37435 48.89884)"
1,1746,1996-10-03 02:00:00+00:00,2023-01-01 01:00:00+00:00,Bd_Macdonald,3630,Bd Macdonald - Rue E. 019,929,Bd_Macdonald - Lounes Matoub,"{'lon': 2.3785952253516314, 'lat': 48.89874752...","LINESTRING (2.37634 48.89868, 2.38085 48.89881)"
2,914,1996-10-03 02:00:00+00:00,2023-01-01 01:00:00+00:00,Av_Pte_Vincennes,524,Pte_Vincennes-Acces_PI,514,Bd_Davout-Pte_de_Vincennes,"{'lon': 2.412529726334607, 'lat': 48.846824834...","LINESTRING (2.41126 48.84690, 2.41152 48.84693..."
3,1708,1996-10-03 02:00:00+00:00,2023-01-01 01:00:00+00:00,Bd_Ney,899,Ney-SNCF_Nord_1,112,Av_Pte_Poissonniers-Bd_Ney,"{'lon': 2.353299882519023, 'lat': 48.898320555...","LINESTRING (2.35254 48.89830, 2.35406 48.89834)"
4,862,1996-10-03 02:00:00+00:00,2023-01-01 01:00:00+00:00,Bd_Massena,497,Bd_Massena_Porte_de_France,495,Bd_Massena-Av-Pte_Vitry,"{'lon': 2.3783630199335652, 'lat': 48.82451111...","LINESTRING (2.37712 48.82405, 2.37960 48.82497)"
...,...,...,...,...,...,...,...,...,...,...
3734,1831,2005-01-01 01:00:00+00:00,2019-06-01 02:00:00+00:00,N_2,942,Pte_Villette-Acces_PI,971,Pte_Villette-Sortie_PE_RN2,"{'lon': 2.3869802636004467, 'lat': 48.90082988...","LINESTRING (2.38707 48.90095, 2.38689 48.90071)"
3735,6140,2005-01-01 01:00:00+00:00,2019-06-01 02:00:00+00:00,SI_Quai_Issy_bretelle_1,599,Quai_Issy_bretelles_1-2,595,SI_Quai_Issy,"{'lon': 2.2701070063099777, 'lat': 48.83548466...","LINESTRING (2.27149 48.83551, 2.27011 48.83552..."
3736,4372,2005-01-01 01:00:00+00:00,2019-06-01 02:00:00+00:00,Av_Marceau,2345,Av_Marceau-Av_Pdt_Wilson,2343,Place_Brisson,"{'lon': 2.2997389284685914, 'lat': 48.86578366...","LINESTRING (2.29958 48.86639, 2.29990 48.86518)"
3737,4931,2005-01-01 01:00:00+00:00,2019-06-01 02:00:00+00:00,Cours_de_Vincennes,289,Vincennes-Picpus-Charonne,2608,Cours_de_Vincennes-Face_35,"{'lon': 2.3999136948032023, 'lat': 48.84808813...","LINESTRING (2.40062 48.84803, 2.39920 48.84815)"


In [6]:
# def process_car_detectors(year, polygon):
#     boundary_gdf = gpd.GeoDataFrame(geometry=[polygon], crs=df_car_detectors.crs)
#     # valid_detectors = df_car_detectors[(df_car_detectors['date_debut'].dt.year <= year) &  (df_car_detectors['date_fin'].dt.year >= year)]
#     # df_car_detectors_without_dupl = valid_detectors.drop_duplicates(
#     # subset='iu_ac', keep='first')
#     car_detectors_within_boundary = gpd.sjoin(
#     df_car_detectors, boundary_gdf, op='within')
#     # car_detectors_within_boundary = car_detectors_within_boundary.rename(
#     # columns={"iu_ac": "detector_id"})
#     return car_detectors_within_boundary

def find_detectors_by_year(df, year):
    """
    Finds detectors installed in a given year.

    Parameters:
    df (DataFrame): GeoDataFrame containing detectors information.
    year (int): The year to filter detectors for.

    Returns:
    DataFrame: DataFrame containing detectors installed in the specified year.
    """
    detectors_in_year = df[(df['date_debut'].dt.year <= year) & (df['date_fin'].dt.year >= year)]
    return detectors_in_year

def compute_changes_in_detectors(df, start_year, end_year):
    """
    Computes changes in detectors between consecutive years.

    Parameters:
    df (DataFrame): GeoDataFrame containing detectors information.
    start_year (int): The start year for analysis.
    end_year (int): The end year for analysis.

    Returns:
    DataFrame: DataFrame containing the count of detectors added and removed each year.
    """
    changes = []
    for year in range(start_year + 1, end_year + 1):
        detectors_prev_year = find_detectors_by_year(df, year - 1)
        detectors_current_year = find_detectors_by_year(df, year)
        
        detectors = detectors_prev_year.intersection(detectors_current_year)
        
        added_detectors = len(detectors_current_year) - len(detectors_prev_year)
        removed_detectors = len(detectors_prev_year) - len(detectors_current_year)
        
        changes.append({'Year': year, 'Detectors_Added': added_detectors, 'Detectors_Removed': removed_detectors})
    
    changes_df = pd.DataFrame(changes)
    return changes_df

def read_detector_data():
    ldd_2010_2012 = pd.read_csv(data_path + 'traffic_data/traffic_data_2010_2012.csv')
    ldd_2013_2020 = pd.read_csv(data_path + 'traffic_data/traffic_data.csv')
    ldd_2021_2022 = pd.read_csv(data_path + 'traffic_data/traffic_data_2021_2022.csv')
    ldd = pd.concat([ldd_2010_2012, ldd_2013_2020, ldd_2021_2022])
    ldd['t_1h'] = pd.to_datetime(ldd['t_1h'])
    return ldd

def find_detectors_by_year_ldd(df, year):
    """
    Finds detectors installed in a given year.

    Parameters:
    df (DataFrame): GeoDataFrame containing detectors information.
    year (int): The year to filter detectors for.

    Returns:
    DataFrame: DataFrame containing detectors installed in the specified year.
    """
    entries_in_year = df[(df['t_1h'].dt.year <= year) & (df['t_1h'].dt.year >= year)]

    det_in_year = entries_in_year['iu_ac'].unique()
    return det_in_year

In [7]:
len(find_detectors_by_year(df_car_detectors, 2000)['iu_ac'].unique())

2876

In [8]:
changes = []
for year in range(1997, 2024):
    detectors_prev_year = find_detectors_by_year(df_car_detectors, year - 1)
    detectors_current_year = find_detectors_by_year(df_car_detectors, year)
    
    unique_det_prev = detectors_prev_year['iu_ac'].unique()
    unique_det_curr = detectors_current_year['iu_ac'].unique()
    
    det_in_both = np.intersect1d(unique_det_prev, unique_det_curr)
    det_in_just_prev = np.setdiff1d(unique_det_prev, det_in_both)
    det_in_just_curr = np.setdiff1d(unique_det_curr, det_in_both)
    
    added_detectors = len(det_in_just_curr) 
    removed_detectors = len(det_in_just_prev) 
    
    print(added_detectors, removed_detectors)
    
    changes.append({'Year': year, 'Number_of_Detectors': len(unique_det_curr), 'Detectors_Added': added_detectors, 'Detectors_Removed': removed_detectors})
    
changes_df = pd.DataFrame(changes)

259 0
47 0
15 0
56 0
57 0
35 0
23 0
3 0
66 0
32 0
37 0
2 0
4 0
0 0
20 0
20 0
11 0
47 7
44 0
4 1
5 13
13 7
49 2
0 15
0 0
0 0
0 0


In [9]:
changes_df

,Year,Number_of_Detectors,Detectors_Added,Detectors_Removed
0,1997,2758,259,0
1,1998,2805,47,0
2,1999,2820,15,0
3,2000,2876,56,0
4,2001,2933,57,0
5,2002,2968,35,0
6,2003,2991,23,0
7,2004,2994,3,0
8,2005,3060,66,0
9,2006,3092,32,0


But how does this look for the traffic data?

In [11]:
ldd = read_detector_data()

In [33]:
def find_detectors_by_year_ldd_delivering_at_least_once_q_or_k(df, year):
    """
    Finds detectors installed in a given year.

    Parameters:
    df (DataFrame): GeoDataFrame containing detectors information.
    year (int): The year to filter detectors for.

    Returns:
    DataFrame: DataFrame containing detectors installed in the specified year.
    """
    entries_in_year = df[(df['t_1h'].dt.year <= year) & (df['t_1h'].dt.year >= year)]
    
    # filter entries_in_year for entries, where q and k are not NaN
    entries_in_year = entries_in_year[(~entries_in_year['q'].isnull()) | (~entries_in_year['k'].isnull())]
    print(entries_in_year.head(50))
    
    det_in_year = entries_in_year['iu_ac'].unique()
    return det_in_year

In [34]:
def find_detectors_by_year_ldd_delivering_at_least_once_q_and_k(df, year):
    """
    Finds detectors installed in a given year.

    Parameters:
    df (DataFrame): GeoDataFrame containing detectors information.
    year (int): The year to filter detectors for.

    Returns:
    DataFrame: DataFrame containing detectors installed in the specified year.
    """
    entries_in_year = df[(df['t_1h'].dt.year <= year) & (df['t_1h'].dt.year >= year)]
    
    # filter entries_in_year for entries, where q and k are not NaN
    entries_in_year = entries_in_year[entries_in_year['q'].notna()]
    entries_in_year = entries_in_year[entries_in_year['k'].notna()]
    print(entries_in_year.head())
    
    det_in_year = entries_in_year['iu_ac'].unique()
    return det_in_year

In [35]:
find_detectors_by_year_ldd_delivering_at_least_once_q_or_k(ldd, 2015)

          iu_ac                t_1h   q        k  hour         day
29977028    799 2015-01-01 05:00:00 NaN  0.43667     5  2015-01-01
29977029    799 2015-01-01 06:00:00 NaN  0.43000     6  2015-01-01
29977030    799 2015-01-01 07:00:00 NaN  0.24722     7  2015-01-01
29977031    799 2015-01-01 08:00:00 NaN  0.19556     8  2015-01-01
29977032    799 2015-01-01 09:00:00 NaN  0.38445     9  2015-01-01
29977033    799 2015-01-01 10:00:00 NaN  0.32334    10  2015-01-01
29977034    799 2015-01-01 11:00:00 NaN  0.49111    11  2015-01-01
29977035    799 2015-01-01 12:00:00 NaN  0.62889    12  2015-01-01
29977036    799 2015-01-01 13:00:00 NaN  0.82500    13  2015-01-01
29977037    799 2015-01-01 14:00:00 NaN  0.66944    14  2015-01-01
29977038    799 2015-01-01 15:00:00 NaN  0.83834    15  2015-01-01
29977039    799 2015-01-01 16:00:00 NaN  1.26722    16  2015-01-01
29977040    799 2015-01-01 17:00:00 NaN  1.28222    17  2015-01-01
29977041    799 2015-01-01 18:00:00 NaN  2.18500    18  2015-0

array([ 799,  651, 4950, ..., 1222, 1111, 1109])

In [ ]:
asdfads

In [14]:
len(find_detectors_by_year_ldd(ldd, 2015))

          iu_ac                t_1h   q        k  hour         day
29977028    799 2015-01-01 05:00:00 NaN  0.43667     5  2015-01-01
29977029    799 2015-01-01 06:00:00 NaN  0.43000     6  2015-01-01
29977030    799 2015-01-01 07:00:00 NaN  0.24722     7  2015-01-01
29977031    799 2015-01-01 08:00:00 NaN  0.19556     8  2015-01-01
29977032    799 2015-01-01 09:00:00 NaN  0.38445     9  2015-01-01


3267

In [36]:
changes = []
for year in range(2010, 2023):
    detectors_prev_year = find_detectors_by_year_ldd_delivering_at_least_once_q_or_k(ldd, year - 1)
    detectors_current_year = find_detectors_by_year_ldd_delivering_at_least_once_q_and_k(ldd, year)
    
    det_in_both = np.intersect1d(detectors_prev_year, detectors_current_year)
    det_in_just_prev = np.setdiff1d(detectors_prev_year, det_in_both)
    det_in_just_curr = np.setdiff1d(detectors_current_year, det_in_both)
    
    added_detectors = len(det_in_just_curr) 
    removed_detectors = len(det_in_just_prev) 
    
    print(added_detectors, removed_detectors)
    
    changes.append({'Year': year, 'Number_of_Detectors': len(detectors_current_year), 'Detectors_Added': added_detectors, 'Detectors_Removed': removed_detectors})
    
changes_ldd_at_least_one_q_or_k_df = pd.DataFrame(changes)

Empty DataFrame
Columns: [iu_ac, t_1h, q, k, hour, day]
Index: []
    iu_ac                t_1h      q    k  hour         day
18    651 2010-01-01 05:00:00  318.0  3.0     5  2010-01-01
19    651 2010-01-01 06:00:00  254.0  3.0     6  2010-01-01
20    651 2010-01-01 07:00:00  233.0  3.0     7  2010-01-01
21    651 2010-01-01 08:00:00  161.0  2.0     8  2010-01-01
22    651 2010-01-01 09:00:00  123.0  1.0     9  2010-01-01
1881 0
    iu_ac                t_1h      q    k  hour         day
0     799 2010-01-01 05:00:00    NaN  1.0     5  2010-01-01
1     799 2010-01-01 06:00:00    NaN  1.0     6  2010-01-01
2     799 2010-01-01 07:00:00    NaN  0.0     7  2010-01-01
3     799 2010-01-01 08:00:00    NaN  0.0     8  2010-01-01
4     799 2010-01-01 09:00:00    NaN  0.0     9  2010-01-01
5     799 2010-01-01 10:00:00    NaN  0.0    10  2010-01-01
6     799 2010-01-01 11:00:00    NaN  0.0    11  2010-01-01
7     799 2010-01-01 12:00:00    NaN  1.0    12  2010-01-01
8     799 2010-01-01 13:00:

In [ ]:
changes_ldd_df

,Year,Number_of_Detectors,Detectors_Added,Detectors_Removed
0,2010,3123,3123,0
1,2011,3143,20,0
2,2012,3167,24,0
3,2013,3179,12,0
4,2014,3230,51,0
5,2015,3267,44,7
6,2016,3271,4,0
7,2017,3268,6,9
8,2018,3262,13,19
9,2019,3310,52,4


In [38]:
changes_ldd_at_least_one_q_or_k_df.to_csv(data_path + 'processed_data/detectors_in_ldd_at_least_one_q_or_k_per_year.csv', index=False)

In [22]:
changes_ldd_at_least_one_q_k_df

,Year,Number_of_Detectors,Detectors_Added,Detectors_Removed
0,2010,1881,1881,0
1,2011,1821,123,183
2,2012,1749,84,156
3,2013,1767,136,118
4,2014,1714,91,144
5,2015,1729,105,90
6,2016,1780,111,60
7,2017,1857,133,56
8,2018,1815,56,98
9,2019,1854,132,93


In [37]:
changes_ldd_at_least_one_q_or_k_df

,Year,Number_of_Detectors,Detectors_Added,Detectors_Removed
0,2010,1881,1881,0
1,2011,1821,83,718
2,2012,1749,40,740
3,2013,1767,79,672
4,2014,1714,33,737
5,2015,1729,34,701
6,2016,1780,59,641
7,2017,1857,47,655
8,2018,1815,24,730
9,2019,1854,61,705
